# Évaluation des Modèles sur Données de Production

In [ ]:
import numpy as np
import pandas as pd
import pickle
import os
from tensorflow import keras
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

## Modèle Machine Learning

In [ ]:
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score

# Charger le modèle ML
model_ml = joblib.load('./models/mushroom_machine_learning.pkl')

# Charger les données
df_eval = pd.read_csv('./dataset/mushroom_eval.csv', sep=';')

print("Shape du dataset:", df_eval.shape)

# Séparer target (dernière colonne)
y_true = df_eval.iloc[:, -1]

# Sélectionner uniquement les features attendues par le modèle
expected_features = model_ml.feature_names_in_
X_eval = df_eval[expected_features]

print(f"Features utilisées: {len(expected_features)}")

# Prédire
y_pred = model_ml.predict(X_eval)

# Accuracy
accuracy_ml = accuracy_score(y_true, y_pred)
print(f"\nAccuracy ML: {accuracy_ml:.4f}")

## Modèle Deep Learning

In [ ]:
# Charger le modèle DL
model_dl = keras.models.load_model('./models/mushroom_deep_learning.keras')

# Fonction de chargement des images
def load_images(directory):
    images = []
    labels = []
    input_shape = model_dl.input_shape[1:3]
    
    for class_name in sorted(os.listdir(directory)):
        class_path = os.path.join(directory, class_name)
        if not os.path.isdir(class_path):
            continue
            
        for img_name in os.listdir(class_path):
            if not img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
                
            img_path = os.path.join(class_path, img_name)
            img = keras.preprocessing.image.load_img(img_path, target_size=input_shape)
            img_array = keras.preprocessing.image.img_to_array(img) / 255.0
            
            images.append(img_array)
            labels.append(class_name)
    
    return np.array(images), np.array(labels)

# Charger les images
X_eval_dl, y_true_dl = load_images('./dataset/eval/')

# Encoder les labels
label_encoder = LabelEncoder()
y_true_encoded = label_encoder.fit_transform(y_true_dl)

# Prédire
y_pred_proba = model_dl.predict(X_eval_dl, verbose=0)
y_pred_dl = np.argmax(y_pred_proba, axis=1)

# Accuracy
accuracy_dl = accuracy_score(y_true_encoded, y_pred_dl)
print(f"Accuracy DL: {accuracy_dl:.4f}")

## Résumé

In [ ]:
print(f"\nModèle ML: {accuracy_ml:.4f}")
print(f"Modèle DL: {accuracy_dl:.4f}")

In [ ]:
import pandas as pd
import joblib
from sklearn.metrics import accuracy_score, confusion_matrix

# Charger le modèle et les données
model_ml = joblib.load('./models/mushroom_machine_learning.pkl')
df_eval = pd.read_csv('./dataset/mushroom_eval.csv', sep=';')

# Préparer les données
y_true = df_eval.iloc[:, -1]
expected_features = model_ml.feature_names_in_
X_eval = df_eval[expected_features]

# Prédire
y_pred = model_ml.predict(X_eval)

# Diagnostics
print("=== DIAGNOSTICS ===")
print(f"Nombre d'exemples: {len(y_true)}")
print(f"Classes uniques dans y_true: {y_true.unique()}")
print(f"Classes uniques dans y_pred: {pd.Series(y_pred).unique()}")
print(f"\nDistribution y_true:\n{y_true.value_counts()}")
print(f"\nDistribution y_pred:\n{pd.Series(y_pred).value_counts()}")
print(f"\nAccuracy: {accuracy_score(y_true, y_pred):.4f}")
print(f"\nMatrice de confusion:\n{confusion_matrix(y_true, y_pred)}")

# Vérifier si les données de test sont identiques aux données d'entraînement
print("\n=== VÉRIFICATION DATA LEAKAGE ===")
print("Premières lignes du dataset d'éval:")
print(df_eval.head())